In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *

In [0]:
# Read data from the Silver table
df_silver_football_matches = spark.read.table("football_matches_catalog.silver.football_matches")

In [0]:
display(df_silver_football_matches)

In [0]:
# Compute statistics for home matches (per season, per team)
df_home_team_stats = (
    df_silver_football_matches
        .withColumnRenamed("HomeTeam", "Team")
        .groupBy("Season", "Team")
        .agg(
            sum("HomeTeamGoals").alias("GoalsScored"),
            sum("AwayTeamGoals").alias("GoalsAgainst"),
            sum(when(col("FinalResult") == "H", 1).otherwise(0)).alias("Win"),
            sum(when(col("FinalResult") == "A", 1).otherwise(0)).alias("Loss"),
            sum(when(col("FinalResult") == "D", 1).otherwise(0)).alias("Tie"),
        )
)

In [0]:
display(df_home_team_stats)

In [0]:
# Compute statistics for away matches (per season, per team)
df_away_team_stats = (
    df_silver_football_matches
        .withColumnRenamed("AwayTeam", "Team")
        .groupBy("Season", "Team")
        .agg(
            sum("AwayTeamGoals").alias("GoalsScored"),
            sum("HomeTeamGoals").alias("GoalsAgainst"),
            sum(when(col("FinalResult") == "A", 1).otherwise(0)).alias("Win"),
            sum(when(col("FinalResult") == "H", 1).otherwise(0)).alias("Loss"),
            sum(when(col("FinalResult") == "D", 1).otherwise(0)).alias("Tie"),
        )
)

In [0]:
display(df_away_team_stats)

In [0]:
# Combine home and away statistics
df_season_team_agg_stats = (
    df_home_team_stats
        .union(df_away_team_stats)
        .groupBy("Season", "Team")
        .agg(
            sum("GoalsScored").alias("GoalsScored"),
            sum("GoalsAgainst").alias("GoalsAgainst"),
            sum("Win").alias("Win"),
            sum("Loss").alias("Loss"),
            sum("Tie").alias("Tie"),
        )
)

In [0]:
display(df_season_team_agg_stats)

In [0]:
# Enrich with derived metrics
df_season_team_enriched_stats = (
    df_season_team_agg_stats
        .withColumn("GoalDifferentials", col("GoalsScored") - col("GoalsAgainst"))
        .withColumn("WinPercentage", round((col("Win") / (col("Win") + col("Loss") + col("Tie"))) * 100, 2))
)

In [0]:
display(df_season_team_enriched_stats)

In [0]:
# Define a window by season, ordering by wins then goal differential
window_spec = (Window.partitionBy("Season").orderBy(desc("Win"), desc("GoalDifferentials")))

# Assign a position for each team in its season
df_gold_team_rankings = (df_season_team_enriched_stats.withColumn("TeamPosition", row_number().over(window_spec)))

In [0]:
display(df_gold_team_rankings)

In [0]:
# Write the final Gold-layer ranking table
df_gold_team_rankings.write.format("delta").mode("overwrite").saveAsTable("football_matches_catalog.gold.football_matches_ranking")